In [1]:
from collections import Counter
from sklearn.externals import joblib
from collections import Counter

import imp
import copy
import pickle
import datetime
import scipy
import numpy as np
import pandas as pd

import utils as my_utils

from sklearn.metrics import silhouette_score, davies_bouldin_score

In [2]:
dataset = pd.read_pickle("datasets/datadf_amazon_musical")

In [3]:
dataset.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,sentiment,summary,unixReviewTime,reviewTime,cleaned,text
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014","[much, write, doe, exactly, supposed, filter, ...",much write doe exactly supposed filter pop sou...
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013","[product, doe, exactly, quite, affordable, rea...",product doe exactly quite affordable realized ...


In [4]:
dataset.shape

(10254, 11)

In [5]:
count_matrix, tfidf_matrix, vocabulary, words = my_utils.processReviews(dataset['text'].values)

In [6]:
barren = np.where(count_matrix.sum(1)==0)[0]

In [7]:
barren

array([], dtype=int64)

### Making Edge_Dict

In [8]:
pickle_in = open("resources/edges_amazon_musical_fasttext_nontrained.pickle","rb")
docs_edges = pickle.load(pickle_in)

In [9]:
len(docs_edges)

10254

In [10]:
docs_edges = np.delete(docs_edges, barren).tolist()

In [11]:
len(docs_edges)

10254

In [13]:
docs_edges[0]

[(1084, 563),
 (1084, 1153),
 (1084, 1939),
 (1084, 798),
 (1084, 1972),
 (1984, 226),
 (1984, 1972),
 (563, 1710),
 (563, 1153),
 (563, 1939),
 (563, 798),
 (1710, 1153),
 (1617, 1397),
 (1617, 394),
 (1397, 1972),
 (1153, 1042),
 (1153, 1939),
 (1153, 798),
 (1153, 1972),
 (1153, 452),
 (1318, 226),
 (1939, 798),
 (1939, 1972),
 (1939, 452)]

In [41]:
edge_dict_m = []
for doc in docs_edges:
    edge_dict = {}
    for i, j in doc:
        try:
            edge_dict[i] += [j]
            edge_dict[j] += [i]
        except:
            edge_dict[i] = [j]
            edge_dict[j] = [i]
    edge_dict_m.append(edge_dict)

In [43]:
edge_dict_m[0]

{226: [1318],
 394: [1617],
 452: [1153, 1939],
 563: [1710, 1153, 1939, 798],
 798: [1084, 563, 1153, 1939],
 1042: [1153],
 1084: [1972],
 1153: [452],
 1318: [226],
 1397: [1617, 1972],
 1617: [394],
 1710: [563, 1153],
 1939: [1084, 563, 1153, 798, 1972, 452],
 1972: [1084, 1984, 1397, 1153, 1939],
 1984: [226, 1972]}

In [35]:
edge_dict

{226: [1318],
 394: [1617],
 452: [1153, 1939],
 563: [1710, 1153, 1939, 798],
 798: [1084, 563, 1153, 1939],
 1042: [1153],
 1084: [1972],
 1153: [452],
 1318: [226],
 1397: [1617, 1972],
 1617: [394],
 1710: [563, 1153],
 1939: [1084, 563, 1153, 798, 1972, 452],
 1972: [1084, 1984, 1397, 1153, 1939],
 1984: [226, 1972]}

In [ ]:
# for i in edge_dict.keys():
#     edge_dict[i] = list(set(edge_dict[i]))

In [ ]:
# len(edge_dict)

In [12]:
lambda_param = 1.0
n_top_words = 5
k = 5

In [ ]:
# topics_grid = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60]

In [ ]:
# folder_name = "baseline_etm_amazon_musical_fasttext_nontrained"

In [ ]:
# folder_name = str(datetime.datetime.now()) + "_" + folder_name
# os.mkdir("dumps/"+folder_name)

In [88]:
import LDA_Init_mods as lda

In [89]:
import imp

In [100]:
imp.reload(lda)

<module 'LDA_Init_mods' from 'LDA_Init_mods.py'>

In [101]:
sampler = lda.LdaSampler(n_topics=5, lambda_param=lambda_param)

In [102]:
%%time
sampler.run(count_matrix, edge_dict_m, maxiter=2)

CPU times: user 28.7 s, sys: 300 ms, total: 29 s
Wall time: 29 s


In [103]:
n_docs, vocab_size = count_matrix.shape

In [104]:
m = 0

In [105]:
def word_indices(vec):
    """
    Turn a document vector of size vocab_size to a sequence
    of word indices. The word indices are between 0 and
    vocab_size-1. The sequence length is equal to the document length.
    """
    for idx in vec.nonzero()[0]:
        for i in xrange(int(vec[idx])):
            yield idx

In [112]:
i = 1
w = 1972

In [113]:
vocab_size = sampler.nzw.shape[1]

In [114]:
%%time
left = (sampler.nzw[:,w] + sampler.beta) / (sampler.nz + sampler.beta * vocab_size)

CPU times: user 73 µs, sys: 0 ns, total: 73 µs
Wall time: 80.8 µs


In [115]:
left

array([0.00803153, 0.00783395, 0.00692729, 0.00811663, 0.00766012])

In [116]:
%%time
right = (sampler.nmz[m,:] + sampler.alpha) / (sampler.nm[m] + sampler.alpha * sampler.n_topics)

CPU times: user 68 µs, sys: 6 µs, total: 74 µs
Wall time: 80.8 µs


In [117]:
right

array([0.00444444, 0.04888889, 0.00444444, 0.67111111, 0.27111111])

In [118]:
%%time
topic_assignment = [0] * sampler.n_topics

CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 18.1 µs


In [119]:
sampler.nzw

array([[ 1.,  1., 17., ...,  8.,  2., 31.],
       [ 6.,  0., 12., ...,  7.,  4.,  8.],
       [12., 14., 53., ...,  5.,  2., 14.],
       [ 3., 18., 23., ..., 25., 19.,  1.],
       [17., 14., 36., ..., 17., 16., 20.]])

In [122]:
edge_dict_m[m][w]

[1084, 1984, 1397, 1153, 1939]

In [123]:
sampler.nzw.shape

(5, 2000)

In [124]:
sampler.phi().shape

(5, 2000)

In [125]:
edge_dict_m[m]

{226: [1318],
 394: [1617],
 452: [1153, 1939],
 563: [1710, 1153, 1939, 798],
 798: [1084, 563, 1153, 1939],
 1042: [1153],
 1084: [1972],
 1153: [452],
 1318: [226],
 1397: [1617, 1972],
 1617: [394],
 1710: [563, 1153],
 1939: [1084, 563, 1153, 798, 1972, 452],
 1972: [1084, 1984, 1397, 1153, 1939],
 1984: [226, 1972]}

In [126]:
C = sampler.phi()[:, edge_dict_m[m][w]]

In [127]:
C.shape

(5, 5)

In [128]:
C

array([[0.20918393, 0.19058824, 0.15128393, 0.21107854, 0.22503477],
       [0.21916646, 0.09647059, 0.23052091, 0.20460932, 0.20587235],
       [0.20519092, 0.23764706, 0.19677183, 0.20076822, 0.18114665],
       [0.18871974, 0.40235294, 0.23639032, 0.20137471, 0.20927214],
       [0.17773896, 0.07294118, 0.18503302, 0.18216921, 0.17867408]])

In [129]:
children = np.eye(C.shape[0])[C.argmax(0)]

In [130]:
children

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [131]:
parent

array([0.19695878, 0.16894758, 0.23297319, 0.1697479 , 0.23137255])

In [132]:
topic_assignment = children.sum(0)
topic_assignment = topic_assignment / count_matrix[m, :].sum()

In [133]:
topic_assignment

array([0.09090909, 0.04545455, 0.        , 0.09090909, 0.        ])

In [135]:
topic_assignment = np.exp(np.dot(sampler.lambda_param, topic_assignment))

In [136]:
topic_assignment

array([1.09516944, 1.04650344, 1.        , 1.09516944, 1.        ])

In [137]:
p_z = np.multiply(left, right) * topic_assignment

In [138]:
p_z

array([3.90928232e-05, 4.00803694e-04, 3.07879487e-05, 5.96556680e-03,
       2.07674395e-03])

In [139]:
np.divide(p_z, p_z.sum())

array([0.00459213, 0.0470814 , 0.00361658, 0.70076003, 0.24394985])

In [ ]:
%%time
left * right * topic_assignment[parent.argmax()]

In [ ]:
left

In [ ]:
right

In [ ]:
topic_assignment

In [ ]:
topic_assignment[parent.argmax()]

In [ ]:
%%time
left * right * topic_assignment[parent.argmax()]

In [ ]:
%%time
np.multiply(left, right) * topic_assignment[parent.argmax()]

In [ ]:
topic_assignment = [0] * sampler.n_topics

In [ ]:
children = []
for i in edge_dict[w]:
    children.append(sampler.nzw[:, i].tolist())
children = np.array(children)
children[children>1] = 1

In [ ]:
children

In [ ]:
for idx, i in enumerate(parent):
    t = 0
    if i>0:
        t =  sum(children[:, idx])
    topic_assignment[idx] = t

In [ ]:
topic_assignment

In [ ]:
if sum(topic_assignment)>0:
    topic_assignment = topic_assignment / sum(topic_assignment)

In [ ]:
topic_assignment

In [ ]:
%%time
sum(topic_assignment)

In [ ]:
topic_assignment = np.exp(np.dot(sampler.lambda_param, topic_assignment))

In [ ]:
topic_assignment

In [ ]:
topic_assignment.sum()

In [ ]:
if sum(topic_assignment)>0:
    topic_assignment = topic_assignment / sum(topic_assignment)

In [ ]:
sum(children[:, idx])

In [ ]:
parent

In [ ]:
enumerate(parent)

In [ ]:
parent

In [ ]:
children.shape

In [ ]:
for idx, i in enumerate(parent):
    print(i)

In [ ]:
topic_assignment

In [ ]:
    def _conditional_distribution(self, m, w, edge_dict):
        """
        Conditional distribution (vector of size n_topics).
        """
        vocab_size = self.nzw.shape[1]
        left = (self.nzw[:,w] + self.beta) / (self.nz + self.beta * vocab_size)
        right = (self.nmz[m,:] + self.alpha) / (self.nm[m] + self.alpha * self.n_topics)
        topic_assignment = [0] * self.n_topics
        parent = self.nzw[:, w]
        try:
            edge_dict[w]
            children = []
            for i in edge_dict[w]:
                children.append(self.nzw[:, i].tolist())
            children = np.array(children)
            children[children>1] = 1
            for idx, i in enumerate(parent):
                t = 0
                if i>0:
                    t =  sum(children[:, idx])
                topic_assignment[idx] = t
            if sum(topic_assignment)>0:
                topic_assignment = topic_assignment / sum(topic_assignment)
            topic_assignment = np.exp(np.dot(self.lambda_param, topic_assignment))
        except:
            topic_assignment = np.exp(topic_assignment)
            topic_assignment = topic_assignment / sum(topic_assignment)
            pass
#         print(parent, parent.argmax())
#         print(topic_assignment)
        p_z = left * right * topic_assignment[parent.argmax()]
        p_z /= np.sum(p_z)
        return p_z

In [ ]:
import multiprocessing

In [ ]:
def multiprocessing_func(k):
    sampler = lda2.LdaSampler(n_topics=k, lambda_param=lambda_param)

    print "Running Sampler...", k
    for it, phi in enumerate(sampler.run(count_matrix, edge_dict)):
        my_utils.print_if_mod(idx, 10)
    joblib.dump(sampler, "dumps/" + folder_name + "/sampler_n_topics_" + str(k))

In [ ]:
pool = multiprocessing.Pool()

In [ ]:
pool.map(multiprocessing_func, topics_grid)

# Evaluations

In [ ]:
folder_name = "2019-08-22 00:47:11.626375_baseline_etm_amazon_fasttext_with_stopwords"

In [ ]:
def multiprocessing_func_eval(k):
    sampler = joblib.load("dumps/"+folder_name + "/sampler_n_topics_" + str(k))

    t_words = sampler.getTopKWords(n_top_words, words)
    top_words = [t_words[i] for i in t_words.keys()]
    dt_distribution = sampler.theta()
    
    print
    print "Topics:", k
    print "Coherance:", my_utils.coherence_score(count_matrix, top_words, vocabulary)
    print "Silhouette Score:", silhouette_score(count_matrix, dt_distribution.argmax(axis=1))
    print "Davies Bouldin Score:", davies_bouldin_score(count_matrix, dt_distribution.argmax(axis=1))

In [ ]:
topics_grid = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60]

In [ ]:
for k in topics_grid:
    multiprocessing_func_eval(k)

# TSNE

In [ ]:
# dt_dist = sampler.theta()

# X_embedded = TSNE(n_components=2).fit_transform(dt_dist)

# import matplotlib.pyplot as plt
# from sklearn.manifold import TSNE

# plt.figure(figsize=(8, 8))
# plt.scatter([i[0] for i in X_embedded], [i[1] for i in X_embedded], c=dt_dist.argmax(axis=1))
# plt.legend(loc=2)
# plt.show()